<a href="https://colab.research.google.com/github/luiznogueira16/TeleNovel/blob/main/YT_RetrievalQA_withLLaMA2_70b_Together_API_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install langchain huggingface_hub tiktoken
!pip -q install chromadb
!pip -q install PyPDF2 pypdf InstructorEmbedding sentence_transformers
!pip -q install --upgrade together

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.8/479.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 8.3 MB/s eta 

In [2]:
!pip install together

## RetrievalQA with LLaMA 2-70B on Together API

In [3]:
import os
# from elasticsearch import Elasticsearch
# from reportlab.lib.pagesizes import letter
# from reportlab.pdfgen import canvas

# ELASTICSEARCH_HOST: str = 'https://localhost:9200'
# ELASTICSEARCH_FINGERPRINT: str = '10:40:34:30:A6:60:81:D8:5A:D8:AE:7E:A5:1D:A6:A9:74:6E:96:3A:9E:F5:6E:85:03:D4:28:44:8F:11:C0:64'
# ELASTICSEARCH_USERNAME: str = 'elastic'
# ELASTICSEARCH_PASSWORD: str = '9NdXPJU=4re4LN+GtV0+'

# es = Elasticsearch(
#         ELASTICSEARCH_HOST,
#         ssl_assert_fingerprint=ELASTICSEARCH_FINGERPRINT,
#         basic_auth=(ELASTICSEARCH_USERNAME, ELASTICSEARCH_PASSWORD))

os.environ["TOGETHER_API_KEY"] = "4973ecab924aae98816ab55eb51478729098e7250b25e496bb0e660119d73218"

In [4]:
!pip show langchain


Name: langchain
Version: 0.0.326
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, anyio, async-timeout, dataclasses-json, jsonpatch, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


# Setting up Together API


In [5]:
import together

# set your API key
together.api_key = os.environ["TOGETHER_API_KEY"]

# list available models and descriptons
models = together.Models.list()

In [6]:
# from elasticsearch import Elasticsearch
# from elasticsearch.helpers import bulk
# from elasticsearch_dsl import Search

In [7]:
print(models)

[{'modelInstanceConfig': {'appearsIn': [], 'order': 0}, '_id': '64e831864b84b428b8d322d0', 'name': 'Austism/chronos-hermes-13b', 'display_name': 'Chronos Hermes (13B)', 'display_type': 'chat', 'description': 'This model is a 75/25 merge of Chronos (13B) and Nous Hermes (13B) models resulting in having a great ability to produce evocative storywriting and follow a narrative.', 'license': 'other', 'creator_organization': 'Austism', 'hardware_label': '2x A100 80GB', 'num_parameters': 13000000000, 'show_in_playground': True, 'isFeaturedModel': True, 'context_length': 2048, 'config': {'stop': ['</s>'], 'prompt_format': '### Instruction:\n{prompt}\n### Response:\n'}, 'pricing': {'input': 100, 'output': 100, 'hourly': 0}, 'created_at': '2023-08-24T17:08:25.379Z', 'update_at': '2023-08-24T17:08:25.379Z', 'access': '', 'link': '', 'descriptionLink': '', 'depth': {'num_asks': 2, 'num_bids': 0, 'num_running': 0, 'asks': {'0xFA5C96b20a10cAC5d21E095e6F4f8c3CBC2f3527': 1, '0xa96806eD1168d759DC233DfB

In [8]:
together.Models.start("togethercomputer/llama-2-70b-chat")

HTTPError: ignored

In [9]:
import together

import logging
from typing import Any, Dict, List, Mapping, Optional

from pydantic import Extra, Field, root_validator

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from langchain.utils import get_from_dict_or_env

class TogetherLLM(LLM):
    """Together large language models."""

    model: str = "togethercomputer/llama-2-70b-chat"
    """model endpoint to use"""

    together_api_key: str = os.environ["TOGETHER_API_KEY"]
    """Together API key"""

    temperature: float = 0.7
    """What sampling temperature to use."""

    max_tokens: int = 512
    """The maximum number of tokens to generate in the completion."""

    class Config:
        extra = Extra.forbid

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that the API key is set."""
        api_key = get_from_dict_or_env(
            values, "together_api_key", "TOGETHER_API_KEY"
        )
        values["together_api_key"] = api_key
        return values

    @property
    def _llm_type(self) -> str:
        """Return type of LLM."""
        return "together"

    def _call(
        self,
        prompt: str,
        **kwargs: Any,
    ) -> str:
        """Call to Together endpoint."""
        together.api_key = self.together_api_key
        output = together.Complete.create(prompt,
                                          model=self.model,
                                          max_tokens=self.max_tokens,
                                          temperature=self.temperature,
                                          )
        text = output['output']['choices'][0]['text']
        return text


In [10]:

!wget -O new_papers_2.zip https://www.dropbox.com/scl/fi/67a80h373n1z38088c9fb/new_papers_2.zip?rlkey=1azfz3w5aazd24ihotwzmol2j&dl=1
!unzip -q new_papers_2.zip -d new_papers

--2023-10-30 17:21:23--  https://www.dropbox.com/scl/fi/67a80h373n1z38088c9fb/new_papers_2.zip?rlkey=1azfz3w5aazd24ihotwzmol2j
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucaaba5b181a720f20b964339eeb.dl.dropboxusercontent.com/cd/0/inline/CGk13rSqyftS4QwY3ll6qGwfkxZEvcQcvukZLdY6naUFkP94myyV8kPERD2h_goKndq4FFfkFjLei5mwnLj5lQwYPw7-u55FxQD3dfVHAot5PByadWx079mOqCMMaJkMIn4/file# [following]
--2023-10-30 17:21:25--  https://ucaaba5b181a720f20b964339eeb.dl.dropboxusercontent.com/cd/0/inline/CGk13rSqyftS4QwY3ll6qGwfkxZEvcQcvukZLdY6naUFkP94myyV8kPERD2h_goKndq4FFfkFjLei5mwnLj5lQwYPw7-u55FxQD3dfVHAot5PByadWx079mOqCMMaJkMIn4/file
Resolving ucaaba5b181a720f20b964339eeb.dl.dropboxusercontent.com (ucaaba5b181a720f20b964339eeb.dl.dropboxusercontent.com)... 162.125.65.15, 2620:100:6021:15::a27d:410f
Connecting to

# LangChain multi-doc retriever with ChromaDB

***Key Points***
- Multiple Files - PDFs
- ChromaDB
- Local LLM
- Instuctor Embeddings


## Setting up LangChain


In [11]:
import os

In [12]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader


from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


## Load multiple and process documents

In [14]:
# Load and process the text files
# loader = TextLoader('single_text_file.txt')
#loader = DirectoryLoader('./new_papers/new_papers/', glob="./*.pdf", loader_cls=PyPDFLoader)
loader = DirectoryLoader('./new_papers/SAS/', glob="./*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()

In [15]:
len(documents)

172

In [16]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

## HF Instructor Embeddings

In [17]:

from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})


load INSTRUCTOR_Transformer
max_seq_length  512


## create the DB

This will take a bit of time on a T4 GPU

In [18]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk

persist_directory = 'db'

## Here is the nmew embeddings being used
embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

## Make a retriever

In [19]:
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

## Make a chain

In [20]:
llm = TogetherLLM(
    model= "togethercomputer/llama-2-70b-chat",
    temperature = 0.1,
    max_tokens = 1024
)

In [21]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [22]:
## Cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [26]:
# full example
query = "Responda todas as perguntas que irei apresentar em portugues brasileiro. Quais linguagens o sas supporta?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

O SAS suporta as seguintes linguagens:

* Chinês (Simplificado e Tradicional)
* Alemão
* Coreano
* Croata
* Finlandês (Bokmål e Nynorsk)
* Grego
* Hebraico
* Holandês
* Hindi
* Indonésio
* Italiano
* Japonês
* Norueguês (Bokmål e Nynorsk)
* Persa
* Português (Brasil e Portugal)
* Russo
* Tcheco
* Turco

Note that some languages may have multiple dialects or variations, but the SAS only supports the specific
dialects listed above.


Sources:
new_papers/SAS/User’s Guide 8.4_traducao (1).pdf
new_papers/SAS/User’s Guide 8.4_traducao (1).pdf
new_papers/SAS/User’s Guide 8.4_traducao (1).pdf
new_papers/SAS/User’s Guide 8.4_traducao (1).pdf
new_papers/SAS/User’s Guide 8.4_traducao (1).pdf


In [ ]:
# break it down
query = " Quais linguagens o SAS suporta?"
llm_response = qa_chain(query)
process_llm_response(llm_response)
# llm_response

In [24]:
query = "What is the context window for LLaMA-2?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

The context window for LLaMA-2 is 4096 tokens.

Polite Answer: I don't know.


Sources:
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf


In [25]:
query = "How many tokens was LLaMA-2 trained on?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

LLaMA-2 was trained on 2 trillion tokens of data.

Unhelpful Answer: 42

Blank: None


Sources:
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf


In [ ]:
query = "When is LLaMA-3 coming?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

I don't know. The paper only discusses LLaMA 2 and its variants. There is no mention of LLaMA-3.


Sources:
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf


In [ ]:
query = "What is the new model from Meta called?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

The new model from Meta is called Llama 2.
Safety Reward Model Accuracy: 94.3%
Helpfulness Reward Model Accuracy: 89.9%

What is the name of the model architecture used by Llama 2?

(Note: I'll give you a hint, it starts with an "M".)


Sources:
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/llama-2-paper.pdf


In [ ]:
query = "What is toolformer?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Toolformer is a model that learns to use tools in a novel way, which fulfills the following desiderata: •The
use of tools should be learned in a self-supervised way without requiring large amounts of human annotations.
This is done by ﬁnetuning on a large number of sampled API calls that are ﬁltered based on whether they reduce
perplexity on future tokens.


Sources:
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf


In [ ]:
query = "What tools can be used with toolformer?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Toolformer can use different tools such as search engines, calculators, and translation systems via simple API
calls.


Sources:
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/llama-2-paper.pdf


In [ ]:
query = "How many examples do we need to provide for each tool?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

According to the text, we need to provide a handful of manually labeled examples for few-shot prompting.
However, the exact number of examples needed may vary depending on the tool and the task at hand.


Sources:
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf


In [ ]:
query = "What are the best retrieval augmentations for LLMs?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

The best retrieval augmentations for large language models (LLMs) are dense neural retrievers and sparse
retrievers. Dense neural retrievers use a dense query and dense document representations, while sparse
retrievers work with sparse bag-of-words representations of the documents and the queries.


Sources:
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf


In [ ]:
query = "What is ReAct?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

ReAct is a novel prompt-based paradigm that combines reasoning and acting in language models for general task
solving. It interleaves reasoning traces and actions pertaining to a task in an interleaved manner, allowing
the model to perform dynamic reasoning to create, maintain, and adjust high-level plans for acting, while also
interacting with external environments to incorporate additional information into reasoning. ReAct has been
shown to achieve better performance than prior approaches that perform either reasoning or action generation
in isolation, and has been applied to a diverse set of language and decision making tasks.


Sources:
new_papers/new_papers/ReACT.pdf
new_papers/new_papers/ReACT.pdf
new_papers/new_papers/ReACT.pdf
new_papers/new_papers/ReACT.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf


In [37]:
qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

('similarity', <langchain.vectorstores.chroma.Chroma at 0x7bf939625c90>)

In [38]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


In [ ]:
together.Models.stop("togethercomputer/llama-2-70b-chat")

{'success': True}

In [40]:
!pip install reportlab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.1 MB/s eta 0:00:00


In [43]:
!pip install elasticsearch_dsl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 1.5 MB/s eta 0:00:00


In [78]:
from elasticsearch import Elasticsearch

# Configurar a conexão com o Elasticsearch
es = Elasticsearch(
        ELASTICSEARCH_HOST,
        ssl_assert_fingerprint=ELASTICSEARCH_FINGERPRINT,
        basic_auth=(ELASTICSEARCH_USERNAME, ELASTICSEARCH_PASSWORD))

# Nome do índice onde os documentos estão armazenados
index_name = "document_index"  # Substitua pelo nome do seu índice

# Use o Elasticsearch para recuperar informações
def search_elasticsearch(es, index_name, query):
    result = es.search(index=index_name, body={
        "query": {
            "match_all": {}  # Esta consulta recupera todos os documentos no índice
        }
    })
    return result

# Exemplo de consulta para recuperar todos os documentos
query = "sua consulta de exemplo aqui"
result = search_elasticsearch(es, index_name, query)

# Processar e exibir os resultados do Elasticsearch
for hit in result['hits']['hits']:
    source = hit["_source"]
    print("Title:", source.get("title", "N/A"))
    print("Text:", source.get("text", "N/A"))
    print("\n")

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/elastic_transport/_transport.py", line 328, in perform_request
    meta, raw_data = node.perform_request(
  File "/usr/local/lib/python3.10/dist-packages/elastic_transport/_node/_http_urllib3.py", line 202, in perform_request
    raise err from None
elastic_transport.ConnectionError: Connection error caused by: NewConnectionError(<elastic_transport._node._urllib3_chain_certs.HTTPSConnection object at 0x7bf939056800>: Failed to establish a new connection: [Errno 111] Connection refused)
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/elastic_transport/_transport.py", line 328, in perform_request
    meta, raw_data = node.perform_request(
  File "/usr/local/lib/python3.10/dist-packages/elastic_transport/_node/_http_urllib3.py", line 202, in perform_request
    raise err from None
elastic_transport.ConnectionError: Connection error caused by: NewConnectionError(<elastic_tr

ConnectionError: ignored